In [33]:
import json
from collections import Counter, defaultdict
from wiki_labels import qid_lab_get
qid_lab_get(42, 'en').keys()

dict_keys(['douglas adams'])

In [88]:
# wiki lang to iso
WIKI_DIR = '/backup/wikidata'
api_key = 'keyVZomJGnZdAB68O'  # TODO save secure
from pyairtable import Table, Api
import sys
sys.path.insert(0, '/projekti/nelma')
from mondoDB.referencedb import provi

if True:
    lang2cc = defaultdict(Counter)
    for k, v in provi.items():
        lang, s, c = v['id'].split('_')
        bod = float(v['Percent'])*3 if 'Percent' in v else 0
        if 'Status' in v:
            if v['Status'] == 'official':
                bod += 1
            elif v['Status'] == 'official_regional':
                bod += 0.5
            elif v['Status'] == 'de_facto_official':
                bod += 0.9
            elif v['Status'] == 'romanized':
                bod += 0.2
            elif v['Status'] == 'foreign':
                bod += -0.5
        if bod>0:
            lang2cc[f'{lang}'][c] = max(bod, lang2cc[f'{lang}'][c])
    api = Api(api_key)
    w2iso = {t['fields']['WMF']: t['fields']['qid'] if 'qid' in t['fields'] else None for t in Table(api_key, 'appUZvAm9EHZgC1Eg', 'wiki_lang').all()}
    wiki2cc = defaultdict(Counter)
    for k, v in w2iso.items():
        if len(k) == 2:
            wiki2cc[v].update(lang2cc[k])
        elif k.count('-') == 1:
            a, b = k.split('-')
            if len(a) == 2 and len(b) == 2:
                wiki2cc[v][b.upper()] += 5 
            elif len(a) == 2 and len(b) == 4:
                #wiki2cc[v].update(lang2cc[f'{k}_{b.uppser()}'])
                wiki2cc[v].update(lang2cc[a])
    wikil2cc = {k: {cc: round(100*v2/max(v.values())) for cc, v2 in v.items()} for k, v in wiki2cc.items() if len(v)>0 and max(v.values())>0}
    json.dump(wikil2cc, open(f'{WIKI_DIR}/wikil2cc.json', 'w'))
else:
    wikil2cc = json.load( open(f'{WIKI_DIR}/wikil2cc.json', 'r'))

wikil2cc['Q1860']['US']

94

In [92]:
# wikipedia to cc
w2cc = {'Q26': 'UK', 'Q31':'BE', 'Q155':'BR', 'Q39':'CH', 'Q1008':'CI', 'Q298':'CL', 'Q148':'CN', 'Q183':'DE', 'Q1005':'GM', 'Q233':'MT', 'Q45':'PT', 'Q878':'AE', 'Q916':'AO', 'Q225':'BA', 'Q398':'BH', 'Q921':'BN', 'Q801':'IL', 'Q189':'IS', 'Q819':'LA', 'Q1028':'MA', 'Q912':'ML', 'Q1183':'PR', 'Q17070':'RE', 'Q1049':'SD', 'Q1044':'SL', 'Q18221':'TC', 'Q869':'TH', 'Q863':'TJ', 'Q686':'VU', 'Q40':'AT', 'Q902':'BD', 'Q784':'DM', 'Q769':'GD', 'Q3769':'GF', 'Q1410':'GI', 'Q28':'HU', 'Q710':'KI', 'Q235':'MC', 'Q236':'ME', 'Q13353':'MS', 'Q664':'NZ', 'Q804':'PA', 'Q685':'SB', 'Q215':'SI', 'Q948':'TN', 'Q962':'BJ', 'Q23635':'BM', 'Q1011':'CV', 'Q191':'EE', 'Q33':'FI', 'Q223':'GL', 'Q41':'GR', 'Q822':'LB', 'Q32':'LU', 'Q709':'MH', 'Q221':'MK', 'Q1032':'NE', 'Q697':'NR', 'Q30971':'PF', 'Q159':'RU', 'Q334':'SG', 'Q192184':'SH', 'Q1045':'SO', 'Q1039':'ST', 'Q678':'TO', 'Q43':'TR', 'Q11703':'VI', 'Q222':'AL', 'Q21590062':'AQ', 'Q967':'BI', 'Q25362':'BL', 'Q917':'BT', 'Q16':'CA', 'Q800':'CR', 'Q142':'FR', 'Q114':'KE', 'Q423':'KP', 'Q884':'KR', 'Q711':'MN', 'Q31057':'NF', 'Q20':'NO', 'Q691':'PG', 'Q219060':'PS', 'Q958':'SS', 'Q16645':'UM', 'Q237':'VA', 'Q757':'VC', 'Q717':'VE', 'Q258':'ZA', 'Q219':'BG', 'Q31063':'CX', 'Q43448':'IO', 'Q760':'LC', 'Q218':'RO', 'Q858':'SY', 'Q672':'TV', 'Q414':'AR', 'Q36004':'CC', 'Q739':'CO', 'Q213':'CZ', 'Q702':'FM', 'Q929':'CF', 'Q262':'DZ', 'Q77':'UY', 'Q977':'DJ', 'Q5785':'KY', 'Q131198':'HM', 'Q232':'KZ', 'Q228':'AD', 'Q115':'ET', 'Q971':'CG', 'Q854':'LK', 'Q26273':'SX', 'Q6250':'EH', 'Q826':'MV', 'Q55':'NL', 'Q29999':'NL', 'Q817':'KW', 'Q38':'IT', 'Q184851':'DG', 'Q963':'BW', 'Q657':'TD', 'Q736':'EC', 'Q790':'HT', 'Q874':'TM', 'Q16644':'MP', 'Q837':'NP', 'Q1246':'XK', 'Q408':'AU', 'Q683':'WS', 'Q842':'OM', 'Q117':'GH', 'Q1006':'GN', 'Q252':'ID', 'Q27':'IE', 'Q805':'YE', 'Q17':'JP', 'Q766':'JM', 'Q986':'ER', 'Q865':'TW', 'Q924':'TZ', 'Q733':'PY', 'Q965':'BF', 'Q79':'EG', 'Q734':'GY', 'Q33788':'NC', 'Q712':'FJ', 'Q954':'ZW', 'Q229':'CY', 'Q794':'IR', 'Q983':'GQ', 'Q126125':'MF', 'Q1020':'MW', 'Q265':'UZ', 'Q230':'GE', 'Q25230':'GG', 'Q419':'PE', 'Q35672':'PN', 'Q851':'SA', 'Q781':'AG', 'Q25228':'AI', 'Q778':'BS', 'Q763':'KN', 'Q37':'LT', 'Q1030':'NA', 'Q34':'SE', 'Q1050':'SZ', 'Q945':'TG', 'Q750':'BO', 'Q242':'BZ', 'Q974':'CD', 'Q29':'ES', 'Q17012':'GP', 'Q846':'QA', 'Q1037':'RW', 'Q14773':'MO', 'Q36823':'TK', 'Q889':'AF', 'Q227':'AZ', 'Q783':'HN', 'Q424':'KH', 'Q347':'LI', 'Q811':'NI', 'Q1041':'SN', 'Q399':'AM', 'Q244':'BB', 'Q8646':'HK', 'Q27561':'BQ', 'Q16635':'GU', 'Q796':'IQ', 'Q785':'JE', 'Q1013':'LS', 'Q1027':'MU', 'Q792':'SV', 'Q25279':'CW', 'Q9648':'FK', 'Q35086':'GS', 'Q224':'HR', 'Q833':'MY', 'Q842829':'SJ', 'Q212':'UA', 'Q881':'VN', 'Q21203':'AW', 'Q23408':'BV', 'Q1007':'GW', 'Q970':'KM', 'Q1029':'MZ', 'Q928':'PH', 'Q695':'PW', 'Q25305':'VG', 'Q17063':'YT', 'Q953':'ZM', 'Q774':'GT', 'Q1014':'LR', 'Q211':'LV', 'Q1042':'SC', 'Q28868874':'EA', 'Q4628':'FO', 'Q9676':'IM', 'Q843':'PK', 'Q214':'SK', 'Q238':'SM', 'Q730':'SR', 'Q754':'TT', 'Q241':'CU', 'Q810':'JO', 'Q217':'MD', 'Q1036':'UG', 'Q34020':'NU', 'Q36':'PL', 'Q813':'KG', 'Q836':'MM', 'Q403':'RS', 'Q1000':'GA', 'Q34617':'PM', 'Q129003':'TF', 'Q574':'TL', 'Q26988':'CK', 'Q5689':'AX', 'Q1025':'MR', 'Q16641':'AS', 'Q184':'BY', 'Q96':'MX', 'Q145':'GB', 'Q668':'IN', 'Q1016':'LY', 'Q35':'DK', 'Q1033':'NG', 'Q35555':'WF', 'Q786':'DO', 'Q1019':'MG', 'Q17054':'MQ', 'Q1009':'CM', 'Q30':'US'}
q2cc = {}
for line in open(f'{WIKI_DIR}/wiki_loc.jsonl'):
    j = json.loads(line)
    for qc in j['country']:
        if qc in w2cc:
            q2cc[j['id']] = w2cc[qc]
            break

print(q2cc['Q494413'], q2cc['Q1435'])




US HR


In [96]:
def get_wiki_cc(country, birth_place, language):
    ccs = Counter()
    for bp in birth_place:
        if bp[5:] in q2cc:
            ccs[q2cc[bp[5:]]] += 3
    for cq in j['country']:
        if cq[5:] in q2cc:
            ccs[q2cc[cq[5:]]] += 1
    for lng in language:
        lng = lng[5:]
        if lng in wikil2cc:
            for cl,bod in wikil2cc[lng].items():
                ccs[cl] += bod / 100
    cc,_ = ccs.most_common()[0] if len(ccs)>0 else (None, None)
    return cc
    

In [110]:
uk = 0
fn = '/projekti/mondonomo/books/thai/thai_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('thai_', 'nonthai_'), 'w')
for i, l in enumerate(open('/backup/wikidata/wiki_person.jsonl')):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'th' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["th"][0].replace(',',' ')
        if cc == 'TH':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head /projekti/mondonomo/books/thai/thai_wiki.csv

Q45209,TH,Poonsawat Kratingdaenggym,พูนสวัสดิ์ กระทิงแดงยิม
Q52006,TH,Pen-Ek Karaket,เป็นเอก การะเกตุ
Q57670,TH,Yingluck Shinawatra,ยิ่งลักษณ์ ชินวัตร
Q74861,TH,Thanin Kraiwichian,ธานินทร์ กรัยวิเชียร
Q78455,TH,Phra Chenduriyang (Piti Wathayakon),พระเจนดุริยางค์ (ปิติ วาทยะกร)
Q158861,TH,Chulalongkorn,พระบาทสมเด็จพระจุลจอมเกล้าเจ้าอยู่หัว
Q243623,TH,Samak Sunthorawet,สมัคร สุนทรเวช
Q284211,TH,Prayad Boonya,ประหยัด บุญญา
Q301290,TH,Phraya Phahonphonphayuhasena (Phot Phahonyothin),พระยาพหลพลพยุหเสนา (พจน์ พหลโยธิน)
Q304412,TH,Narisara Nuwattiwong,สมเด็จพระเจ้าบรมวงศ์เธอ เจ้าฟ้ากรมพระยานริศรานุวัดติวงศ์


In [114]:
from tqdm.notebook import tqdm
uk = 0
fn = '/backup/wikidata/hr_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('hr_', 'nonhr_'), 'w')
for l in tqdm(open('/backup/wikidata/wiki_person.jsonl'), total=10_044_571):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'hr' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["hr"][0].replace(',',' ')
        if cc == 'HR':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head $fn

  0%|          | 0/10044571 [00:00<?, ?it/s]

Q56220,HR,Ivo Sanader,Ivo Sanader
Q71774,HR,Igor Bubnjić,Igor Bubnjić
Q73376,HR,Wilhelm Ehm,Wilhelm Ehm
Q92358,HR,Tom Bunk,Tom Bunk
Q104440,HR,Vladimir Milojčić,Vladimir Milojčić
Q112081,HR,Archduke Wilhelm of Austria,Nadvojvoda Vilim Austrijski
Q114408,HR,Adolf Mussafia,Adolfo Mussafia
Q143424,HR,Vedran Mataija,Vedran Mataija
Q149090,HR,Ivan Ribar,Ivan Ribar
Q152506,HR,Nina Badrić,Nina Badrić


In [115]:
!wc -l  $fn

6627 /backup/wikidata/hr_wiki.csv
